<a href="https://colab.research.google.com/github/LiibanMo/LiibanMo/blob/main/Linear_Models.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [136]:
from abc import ABC, abstractmethod
from typing_extensions import Self, Optional

import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

In [137]:
RANDOM_STATE = 123
N = 100
P = 3
INTERCEPT = 3
LAMBDA = 0.5

In [138]:
class Linear_Model(ABC):
    @abstractmethod
    def fit(self, X:np.ndarray, y:np.ndarray) -> Self:
        pass

    @abstractmethod
    def predict(self, X:np.ndarray) -> np.ndarray:
        pass

In [139]:
class LinearRegression(Linear_Model):
    def __init__(self, fit_intercept:bool=True) -> None:
        self.fit_intercept:bool = fit_intercept

    def fit(self, X:np.ndarray, y:np.ndarray) -> Self:
        if self.fit_intercept:
            self.X = np.column_stack((np.ones(X.shape[0]), X))
        else:
            self.X = X.copy()

        self.y = y.copy()

        self.rank_ = np.linalg.matrix_rank(self.X)

        if self.rank_ == np.min(self.X.shape):
            self.coef_ = np.linalg.solve(self.X.T @ self.X, self.X.T @ self.y)
        else:
            raise ValueError(f"Design matrix is not full rank. Has rank {self.rank_}. Should have rank {np.min(self.X.shape)}.")

        return self

    def predict(self, X:np.ndarray) -> np.ndarray:
        return X @ self.coef_[1:] + self.coef_[0]

In [140]:
class Ridge(Linear_Model):
    def __init__(self, lambda_:float=1, fit_intercept:bool=True):
        self.lambda_ = lambda_
        self.fit_intercept = fit_intercept

    def fit(self, X:np.ndarray, y:np.ndarray) -> Self:
        if self.fit_intercept:
            self.X = np.column_stack((np.ones(X.shape[0]), X))
        else:
            self.X = X.copy()

        self.y = y.copy()

        LHS = self.X.T @ self.X + self.lambda_ * np.eye(self.X.shape[1])
        RHS = self.X.T @ self.y

        if np.linalg.det(LHS) != 0:
            self.coef_ = np.linalg.solve(LHS, RHS)
        else:
            raise ValueError("LHS of Normal Equation is not invertible.")

        return self

    def predict(self, X:np.ndarray) -> np.ndarray:
        return X @ self.coef_[1:] + self.coef_[0]

In [141]:
np.random.seed(RANDOM_STATE)

coef = np.random.rand(P)
X = np.random.rand(N, P)
y = INTERCEPT + X @ coef + np.random.normal(0, 1, size=(N))

In [142]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=True, random_state=RANDOM_STATE)

In [143]:
def evaluate_model(model_:Linear_Model, X_train:np.ndarray, X_test:np.ndarray, y_train:np.ndarray, y_test:np.ndarray, lambda_:Optional[float]=None):
    if lambda_:
        model = model_(lambda_)
    else:
        model = model_()

    fitted_model = model.fit(X_train, y_train)
    y_pred = fitted_model.predict(X_test)
    mse = mean_squared_error(y_test, y_pred)

    return mse

In [144]:
evaluate_model(LinearRegression, X_train, X_test, y_train, y_test)

1.0976843875914413

In [145]:
evaluate_model(Ridge, X_train, X_test, y_train, y_test, lambda_ = LAMBDA)

1.0995414457855548